adapted from: https://www.kaggle.com/code/kanncaa1/recurrent-neural-network-with-pytorch

In [9]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tweedejaars_project import load_df

In [25]:
""" Prepare Dataset """

# load data
df = load_df()
# highly_correlated_features = ['naive_strategy_action', 'min_ptu_price_known',
#                               'min_price_published', 'forecast_wind',
#                               'time_since_last_two_sided',
#                               'two_sided_daily_count', 'mid_price_published',
#                               'vwap_avg', 'vwap_std', 'vwap_qty_sum',
#                               'minute_in_ptu', 'downward_dispatch_published',
#                               'settlement_price_bestguess', 'import_capacity',
#                               'upward_dispatch_published',
#                               'settlement_price_realized', 'forecast_solar']

highly_correlated_features = ['naive_strategy_action',
                              'time_since_last_two_sided',
                              'two_sided_daily_count', 'mid_price_published',
                              'vwap_avg', 'vwap_std', 'vwap_qty_sum',
                              'minute_in_ptu']


# split data into features and target
targets_numpy = df['target_two_sided_ptu']
features_numpy = df[highly_correlated_features]

# train test split
# could be replaced with scikit-learn TimeSeriesSplit
X_train, X_test = np.split(features_numpy, [int(.7 * len(features_numpy))])
y_train, y_test = np.split(targets_numpy, [int(.7 * len(targets_numpy))])

# create feature and targets tensor for train and test set.
featuresTrain = torch.Tensor(X_train.to_numpy(dtype=np.float64))
targetsTrain = torch.Tensor(y_train.to_numpy(dtype=np.float64))

featuresTest = torch.Tensor(X_test.to_numpy(dtype=np.float64))
targetsTest = torch.Tensor(y_test.to_numpy(dtype=np.float64))

# force the right dimension
targetsTrain = targetsTrain.unsqueeze(1)
targetsTest = targetsTest.unsqueeze(1)

# Pytorch train and test sets
train = torch.cat((featuresTrain, targetsTrain), dim=1)
test = torch.cat((featuresTest, targetsTest), dim=1)

print(test.shape)

# batch_size, epoch and iteration
first_batch_size = 15
n_iters = 10000
num_epochs = 1 + int(n_iters / (len(X_train) / first_batch_size))

# data loader makes batches and iterable
train_loader = DataLoader(train, batch_size = first_batch_size, shuffle=False)
test_loader = DataLoader(test, batch_size = first_batch_size, shuffle=False)

torch.Size([66074, 9])


/home/tycho/miniconda3/envs/tweedejaars_project/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/tycho/miniconda3/envs/tweedejaars_project/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [23]:
""" Create RNN Model """

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(RNNModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True,
                          nonlinearity='relu')
        
        # Collection layer
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Range redistribution [0,1]
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_dim,
                         requires_grad=True)
        
        try:
            h0 = hn
        except:
            pass

        # One time step
        rnn_out, hn = self.rnn(x, h0)
        
        fc_out = self.fc(rnn_out[:, -1])  # last output

        output = self.sigmoid(fc_out)

        return output


# Create RNN
input_dim =  8    # input dimension
hidden_dim = 60   # hidden layer dimension
num_layers = 2    # number of hidden layers
output_dim = 15   # output dimension
seq_length = 15   # the number of time steps in each sequence

model = RNNModel(input_dim, hidden_dim, num_layers, output_dim)

# SGD Optimizer
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [26]:
""" Run model """

batch_size = int(first_batch_size / seq_length) 
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
error = nn.BCEWithLogitsLoss()
test_iter = iter(test_loader)

for epoch in range(num_epochs):
    for batch in train_loader:
        
        # Stop if the (last) batch is not big enough
        if batch.shape[0] != 15:
            break
        
        train = batch[:, :-1].reshape((batch_size, seq_length, input_dim))
        targets = batch[:, -1:]

        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        model.train()
        outputs = model(train)
        
        # Calculate softmax and cross entropy loss
        loss = error(outputs.T, targets)

        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        # Calculate Accuracy
        if count % 100 == 0:

            correct = 0

            # Iterate through 200 batches in test dataset
            for _ in range(200):

                # load a new batch
                batch = next(test_iter)

                # the iter will eventually run out, reload it              
                if True in torch.isnan(batch):
                    test_iter = iter(test_loader)
                    batch = next(test_iter)
            
                # seperate features and target
                test_features = batch[:, :-1].reshape((batch_size, seq_length, input_dim))
                test_targets = batch[:,-1:]
                
                # Forward propagation
                outputs = model(test_features)
                
                # Round predictions to get final prediction
                predicted = torch.round(outputs.T)
                
                # print(predicted, test_targets)
                correct += (predicted == test_targets).sum()
                
            total = batch.shape[0]*200
            accuracy = (correct / float(total)) * 100
            positives = predicted.sum()
                
            # store loss and iteration
            iteration_list.append(count)
            loss_list.append(loss)
            accuracy_list.append(accuracy)

            # Print results
            print(f'Iteration: {count}  Loss: {loss}  Accuracy: {accuracy} % N_postive: {positives}')

Iteration: 100  Loss: 0.8959591388702393  Accuracy: 71.0 % N_postive: 4.0
Iteration: 200  Loss: 0.9403648376464844  Accuracy: 82.53333282470703 % N_postive: 3.0
Iteration: 300  Loss: 0.5704194903373718  Accuracy: 83.36666870117188 % N_postive: 3.0
Iteration: 400  Loss: 0.8080807328224182  Accuracy: 88.33333587646484 % N_postive: 2.0
Iteration: 500  Loss: 0.6989461779594421  Accuracy: 94.9000015258789 % N_postive: 1.0
Iteration: 600  Loss: 0.7033393979072571  Accuracy: 98.23333740234375 % N_postive: 0.0
Iteration: 700  Loss: 0.6939347982406616  Accuracy: 98.53333282470703 % N_postive: 0.0
Iteration: 800  Loss: 0.6994175314903259  Accuracy: 99.66666412353516 % N_postive: 0.0
Iteration: 900  Loss: 0.6961520910263062  Accuracy: 95.76667022705078 % N_postive: 0.0
Iteration: 1000  Loss: 0.6934558749198914  Accuracy: 99.96666717529297 % N_postive: 0.0
Iteration: 1100  Loss: 0.694873034954071  Accuracy: 99.66666412353516 % N_postive: 0.0
Iteration: 1200  Loss: 0.6936126351356506  Accuracy: 99.

In [ ]:
""" Model Evaluation """

' Model Evaluation '